In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

# Load labeled dataset
df = pd.read_csv("labeled_thermal_features.csv")

# Separate features (X) and labels (y)
X = df.drop(columns=["Frame", "Porosity Label"])  # Exclude 'Frame' (not useful for training)
y = df["Porosity Label"]

# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Apply SMOTE to balance dataset
smote = SMOTE(sampling_strategy=0.2, random_state=42)  # Increase anomalies to 20% of normal samples
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=200,  # Number of trees
    max_depth=5,  # Limit depth to avoid overfitting
    min_samples_split=5,  # Minimum samples per split
    min_samples_leaf=2,  # Minimum samples per leaf
    max_features="sqrt",  # Random subset of features per tree
    class_weight="balanced",  # Handle class imbalance
    random_state=42,
    n_jobs=-1  # Use all available CPU cores
)

# Train the model
rf_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print evaluation results
print("\nRandom Forest Model Evaluation:")
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

# Save the trained model
with open("random_forest_anomaly_model.pkl", "wb") as model_file:
    pickle.dump(rf_classifier, model_file)

print("✅ Random Forest model saved as 'random_forest_anomaly_model.pkl'.")



Random Forest Model Evaluation:
Accuracy: 0.99
Confusion Matrix:
[[298   1]
 [  3  11]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       299
           1       0.92      0.79      0.85        14

    accuracy                           0.99       313
   macro avg       0.95      0.89      0.92       313
weighted avg       0.99      0.99      0.99       313

✅ Random Forest model saved as 'random_forest_anomaly_model.pkl'.


In [2]:
import matplotlib.pyplot as plt
feature_importance = rf_classifier.feature_importances_
feature_names = X_train.columns

plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_importance)
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Random Forest Feature Importance")
plt.show()
